In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [109]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1 ,4, 3 , padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(4),
            nn.Dropout(0.1),

            nn.Conv2d(4 ,8, 3 , padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1),

            # nn.Conv2d(4 ,4, 3 , padding = 1),
            # nn.ReLU(),
            # nn.BatchNorm2d(4),
            # nn.Dropout(0.1),

            nn.MaxPool2d(2,2),
            nn.Conv2d(8, 8, 1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(8 ,16, 3 , padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),

            nn.Conv2d(16 ,16, 3 , padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),

            nn.Conv2d(16 ,16, 3 , padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),

            nn.MaxPool2d(2,2),
            nn.Conv2d(16, 16, 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1)
        )
      
        self.conv3 = nn.Sequential(
            nn.Conv2d(16 ,32, 3 , padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            nn.Conv2d(32 ,16, 3 , padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),

            # nn.Conv2d(32 ,32, 3 , padding = 1),
            # nn.ReLU(),
            # nn.BatchNorm2d(32),
            # nn.Dropout(0.1),

            nn.Conv2d(16, 16, 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1)


        )

        self.fc1 = nn.Sequential(
            nn.AvgPool2d(kernel_size = 7)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features = 16, out_features = 10)
        )






    def forward(self, x):

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc1(x)

        x = x.view(x.size(0), -1)
        x = self.fc2(x)


        return F.log_softmax(x)

In [110]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
              ReLU-2            [-1, 4, 28, 28]               0
       BatchNorm2d-3            [-1, 4, 28, 28]               8
           Dropout-4            [-1, 4, 28, 28]               0
            Conv2d-5            [-1, 8, 28, 28]             296
              ReLU-6            [-1, 8, 28, 28]               0
       BatchNorm2d-7            [-1, 8, 28, 28]              16
           Dropout-8            [-1, 8, 28, 28]               0
         MaxPool2d-9            [-1, 8, 14, 14]               0
           Conv2d-10            [-1, 8, 14, 14]              72
             ReLU-11            [-1, 8, 14, 14]               0
      BatchNorm2d-12            [-1, 8, 14, 14]              16
    

<ipython-input-109-8c328c37ef63>:263: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [111]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [112]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [113]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-109-8c328c37ef63>:263: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.16144181787967682 batch_id=937: 100%|██████████| 938/938 [00:26<00:00, 35.91it/s]



Test set: Average loss: 0.1134, Accuracy: 9643/10000 (96%)



loss=0.08557406812906265 batch_id=937: 100%|██████████| 938/938 [00:23<00:00, 40.33it/s]



Test set: Average loss: 0.1116, Accuracy: 9644/10000 (96%)



loss=0.18140625953674316 batch_id=937: 100%|██████████| 938/938 [00:23<00:00, 40.54it/s]


...


Test set: Average loss: 0.0315, Accuracy: 9895/10000 (99%)



loss=0.13735821843147278 batch_id=937: 100%|██████████| 938/938 [00:23<00:00, 40.06it/s]



Test set: Average loss: 0.0231, Accuracy: 9929/10000 (99%)



loss=0.2238200455904007 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 41.03it/s]



Test set: Average loss: 0.0180, Accuracy: 9947/10000 (99%)

